In [77]:
import numpy as np
import librosa
import scipy.signal as signal
import IPython.display as ipd

# Load the impulse response file
ir_file = "/content/sample_data/PlateSmall_01.wav"
impulse_response, fs = librosa.load(ir_file, sr=None, mono=True)

# Load the input audio file
input_file = "/content/sample_data/blues_guitar.wav"
input_audio, fs = librosa.load(input_file, sr=None, mono=True)

# Set buffer size and initialize buffer and output arrays
buffer_size = len(impulse_response) * 4
num_buffers = int(np.ceil(len(input_audio) / buffer_size))
buffer = np.zeros((num_buffers, buffer_size + len(impulse_response) - 1))
output = np.zeros(len(input_audio) + len(impulse_response) - 1)

# Initialize index variables
ir_index = 0

# Calculate FFT of the impulse response
ir_fft = np.fft.rfft(impulse_response, n=len(buffer[0]))

# Process audio in blocks of buffer_size samples
for i in range(num_buffers):
    # Get input samples from audio file
    start_idx = i * buffer_size
    end_idx = start_idx + buffer_size
    input_samples = input_audio[start_idx:end_idx]

    # Update buffer
    buffer[i] = np.roll(buffer[i], -len(input_samples))
    buffer[i][-len(input_samples):] = input_samples

    # Calculate FFT of the buffer
    buffer_fft = np.fft.rfft(buffer[i], n=len(buffer[i]))

    # Multiply FFT of buffer with FFT of impulse response
    output_fft = buffer_fft * ir_fft

    # Calculate inverse FFT of output
    output_samples = np.fft.irfft(output_fft)

    # Add output samples to output array
    output_start_idx = i * buffer_size + ir_index
    output_end_idx = output_start_idx + buffer_size
    output[output_start_idx:output_end_idx] += output_samples[ir_index:ir_index+buffer_size]

    # Increment index variables
    ir_index += buffer_size

    # Check if end of impulse response has been reached
    if ir_index >= len(impulse_response):
        ir_index = 0

# Normalize output
output /= np.max(np.abs(output))

# Display output
ipd.display(ipd.Audio(data=output, rate=fs))

# Save output to file
#output_file = "/content/sample_data/output_v1.wav"
#librosa.output.write_wav(output_file, output, fs)
